<a href="https://colab.research.google.com/github/solomontessema/Agentic-AI-with-Python/blob/main/notebooks/Task%20Flow%20with%20LangGraph/Embedding_an_Agent_Node.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table>
  <tr>
    <td><img src="https://ionnova.com/img/ionnova_logo_name_2.png" width="120px"></td>
    <td><h1>Embedding an Agent Node</h1></td>
  </tr>
</table>

In [ ]:
!pip install -qU langgraph langchain langchain-openai python-dotenv

In [49]:

from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from langgraph.graph import StateGraph, END
from typing import TypedDict
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def get_weather(city):
    return f"Simulated weather for {city}: Sunny, 24°C."

search_tool = Tool(
    name="get_weather",
    func=get_weather,
    description="Fetch simulated weather data for a given city."
)

agent = create_agent(
    model=llm,
    tools=[search_tool],
    system_prompt="You are a helpful assistant. You must use the provided tools"
)

# Create Graph Node That Runs the Agent
class GraphState(TypedDict):
    input: str
    agent_response: str

# Agent node
def agent_node(state: GraphState):
    print("[Agent Node] Invoking LangChain agent...")
    response=agent.invoke(
        {"messages": [{"role": "user", "content": state["input"]}]}
        )
    result = response["messages"][-1].content
    state["agent_response"] = result
    return state

# Graph setup
graph = StateGraph(GraphState)
graph.add_node("agent", agent_node)
graph.set_entry_point("agent")
graph.set_finish_point("agent")
graph = graph.compile()


In [51]:
output = graph.invoke({"input": "What is the weather in Nairobi."})
print("Agent Output:", output["agent_response"])

[Agent Node] Invoking LangChain agent...
Agent Output: The weather in Nairobi is sunny with a temperature of 24°C.
